In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import us
import folium
from folium import plugins
import time
from geopy.geocoders import Nominatim

# Historic Data Exploration

## Natural Disasters

In [2]:
# summary of all states (combined from map_data above)

nat_dis = pd.read_csv('./../Natural_Disaster_Data/DisasterDeclarationsSummaries.csv')
print(nat_dis.shape)
nat_dis.head(3)

(58771, 19)


,disasterNumber,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,state,declarationDate,fyDeclared,disasterType,incidentType,title,incidentBeginDate,incidentEndDate,disasterCloseOutDate,declaredCountyArea,placeCode,hash,lastRefresh,id
0,1,0,1,1,1,GA,1953-05-02T00:00:00.000Z,1953,DR,Tornado,TORNADO,1953-05-02T00:00:00.000Z,1953-05-02T00:00:00.000Z,1954-06-01T00:00:00.000Z,NaN,NaN,1dcb40d0664d22d39de787b706b0fa69,2019-07-26T18:08:57.368Z,5d1bbd8c8bdcfa6efb32fd8d
1,2,0,1,1,1,TX,1953-05-15T00:00:00.000Z,1953,DR,Tornado,TORNADO & HEAVY RAINFALL,1953-05-15T00:00:00.000Z,1953-05-15T00:00:00.000Z,1958-01-01T00:00:00.000Z,NaN,NaN,61612cea5779e361b429799098974b6a,2019-07-26T18:08:57.370Z,5d1bbd8c8bdcfa6efb32fd8e
2,3,0,1,1,1,LA,1953-05-29T00:00:00.000Z,1953,DR,Flood,FLOOD,1953-05-29T00:00:00.000Z,1953-05-29T00:00:00.000Z,1960-02-01T00:00:00.000Z,NaN,NaN,86f3e47785cb7acc51364d4535d36101,2019-07-26T18:08:57.369Z,5d1bbd8c8bdcfa6efb32fd8f


In [3]:
nat_dis.columns

Index(['disasterNumber', 'ihProgramDeclared', 'iaProgramDeclared',
       'paProgramDeclared', 'hmProgramDeclared', 'state', 'declarationDate',
       'fyDeclared', 'disasterType', 'incidentType', 'title',
       'incidentBeginDate', 'incidentEndDate', 'disasterCloseOutDate',
       'declaredCountyArea', 'placeCode', 'hash', 'lastRefresh', 'id'],
      dtype='object')

In [4]:
nat_dis.isnull().sum().sort_values(ascending = False)

disasterCloseOutDate    16569
incidentEndDate          8033
placeCode                 235
declaredCountyArea        235
hash                       73
lastRefresh                73
id                          0
state                       0
ihProgramDeclared           0
iaProgramDeclared           0
paProgramDeclared           0
hmProgramDeclared           0
incidentType                0
declarationDate             0
fyDeclared                  0
disasterType                0
title                       0
incidentBeginDate           0
disasterNumber              0
dtype: int64

In [5]:
pd.set_option('display.max_rows', 500)
pd.DataFrame(nat_dis.groupby('state')['incidentType'].value_counts())

incidentType
state incidentType                  
AK    Biological                  71
      Severe Storm(s)             64
      Flood                       47
      Fire                        21
      Freezing                    14
      Earthquake                  13
      Other                        4
      Snow                         4
      Coastal Storm                2
AL    Severe Storm(s)            545
      Hurricane                  507
      Biological                 138
      Flood                      104
      Drought                     67
      Snow                        67
      Tornado                     62
      Fire                        11
AR    Severe Storm(s)            635
      Flood                      272
      Severe Ice Storm           207
      Biological                 152
      Tornado                     95
      Hurricane                   93
      Drought                     32
      Snow                        17
AS    Hurricane                   25
      Typhoon                     18
      Biological                  10
      Severe Storm(s)              7
      Drought                      5
      Earthquake                   5
      Flood                        5
AZ    Flood                       78
      Biological                  74
      Severe Storm(s)             67
      Fire                        57
      Hurricane                   15
      Drought                      8
      Freezing                     1
CA    Fire                       363
      Severe Storm(s)            325
      Flood                      320
      Biological                 116
      Hurricane                   58
      Freezing                    57
      Drought                     47
      Coastal Storm               43
      Earthquake                  31
      Dam/Levee Break              5
      Fishing Losses               4
      Tsunami                      3
      Other                        1
CO    Flood                      146
      Biological                 133
      Fire                       132
      Coastal Storm               64
      Snow                        55
      Drought                     42
      Severe Storm(s)             28
      Tornado                     27
      Dam/Levee Break              1
CT    Snow                        60
      Hurricane                   59
      Severe Storm(s)             56
      Biological                  21
      Flood                       17
      Tornado                      5
DC    Severe Storm(s)              9
      Hurricane                    6
      Snow                         3
      Biological                   2
      Earthquake                   1
      Other                        1
DE    Hurricane                   19
      Snow                        13
      Biological                   6
      Severe Storm(s)              5
      Drought                      3
      Flood                        3
      Coastal Storm                1
FL    Hurricane                  930
      Severe Storm(s)            326
      Fire                       254
      Freezing                   147
      Biological                 144
      Flood                       73
      Tornado                     38
      Coastal Storm               25
      Human Cause                  4
      Other                        1
FM    Typhoon                     23
      Drought                      6
      Fire                         1
      Flood                        1
GA    Hurricane                  884
      Severe Storm(s)            404
      Biological                 321
      Tornado                    157
      Severe Ice Storm           152
      Drought                    144
      Snow                        93
      Flood                       62
      Fire                        17
      Freezing                     6
GU    Typhoon                     13
      Biological                   2
      Severe Storm(s)       

In [6]:
nat_dis['declarationDate'] = pd.to_datetime(nat_dis['declarationDate'])
nat_dis['incidentBeginDate'] = pd.to_datetime(nat_dis['incidentBeginDate'])
nat_dis['incidentEndDate'] = pd.to_datetime(nat_dis['incidentEndDate'])

In [7]:
nat_dis['beginMonth'] = nat_dis['incidentBeginDate'].dt.month
nat_dis['beginYear'] = nat_dis['incidentBeginDate'].dt.year
nat_dis['beginDay'] = nat_dis['incidentBeginDate'].dt.day

In [8]:
nat_dis['beginMonth'].value_counts()

1     12483
8      6923
9      6589
4      5026
5      4616
3      4480
6      4116
2      3759
12     3755
10     3064
7      2299
11     1661
Name: beginMonth, dtype: int64

In [9]:
nat_dis[nat_dis['beginYear'] == 2020][['incidentBeginDate', 'disasterType', 'incidentType', 'state', 'declaredCountyArea']].isnull().sum()

incidentBeginDate      0
disasterType           0
incidentType           0
state                  0
declaredCountyArea    21
dtype: int64

In [10]:
nat_dis_2020 = nat_dis[(nat_dis['beginYear'] == 2020) & (nat_dis['incidentType'] != 'Biological')][['disasterType', 'incidentType', 'declaredCountyArea', 'state', 'beginYear', 'beginMonth', 'beginDay', 'incidentBeginDate']]
nat_dis_2020.dropna(inplace = True)

counties_2020 = []
for county in nat_dis_2020['declaredCountyArea']:
    if county.endswith('(County)'):
        counties_2020.append(county[:-9])
    else:
        counties_2020.append(county)

In [11]:
nat_dis_2020['declaredCountyArea'] = counties_2020

In [12]:
nat_dis_2020.drop(index = 57357, inplace = True)

In [13]:
nat_dis_2020['address'] = nat_dis_2020['declaredCountyArea'] + ' County, ' + nat_dis_2020['state']

In [14]:
geolocator = Nominatim(user_agent="griffin_app_1")
address_latlong = []

for address in nat_dis_2020['address']:
    location = geolocator.geocode(address)
    address_latlong.append([address, location.latitude, location.longitude])
    time.sleep(1)
    
address_latlong[:5]

[['Wilson County, TN', 36.1466658, -86.3119998],
 ['Putnam County, TN', 36.1379872, -85.4553784],
 ['Davidson County, TN', 36.189724, -86.7857862],
 ['Beaver County, OK', 36.7621116, -100.4904955],
 ['Milwaukee County, WI', 43.018032500000004, -87.97408733038324]]

In [15]:
address_latlong = pd.DataFrame(address_latlong)

In [16]:
address_latlong.columns = ['address', 'latitude', 'longitude']

In [17]:
nat_dis_2020_latlong = pd.merge(nat_dis_2020, address_latlong, on = 'address')

In [18]:
nat_dis_2020_latlong.head(5)

,disasterType,incidentType,declaredCountyArea,state,beginYear,beginMonth,beginDay,incidentBeginDate,address,latitude,longitude
0,DR,Tornado,Wilson,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Wilson County, TN",36.146666,-86.312000
1,DR,Tornado,Putnam,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Putnam County, TN",36.137987,-85.455378
2,DR,Tornado,Davidson,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Davidson County, TN",36.189724,-86.785786
3,FM,Fire,Beaver,OK,2020,3,7,2020-03-07 08:00:00+00:00,"Beaver County, OK",36.762112,-100.490495
4,DR,Flood,Milwaukee,WI,2020,1,10,2020-01-10 00:00:00+00:00,"Milwaukee County, WI",43.018033,-87.974087


In [19]:
nat_dis_2020_latlong.state.value_counts()

MS    29
SC    27
KY    27
WA    15
TN    14
WI     3
OR     3
OK     1
Name: state, dtype: int64

In [20]:
nat_dis_2020_latlong.head()

,disasterType,incidentType,declaredCountyArea,state,beginYear,beginMonth,beginDay,incidentBeginDate,address,latitude,longitude
0,DR,Tornado,Wilson,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Wilson County, TN",36.146666,-86.312000
1,DR,Tornado,Putnam,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Putnam County, TN",36.137987,-85.455378
2,DR,Tornado,Davidson,TN,2020,3,3,2020-03-03 00:01:00+00:00,"Davidson County, TN",36.189724,-86.785786
3,FM,Fire,Beaver,OK,2020,3,7,2020-03-07 08:00:00+00:00,"Beaver County, OK",36.762112,-100.490495
4,DR,Flood,Milwaukee,WI,2020,1,10,2020-01-10 00:00:00+00:00,"Milwaukee County, WI",43.018033,-87.974087


In [21]:
nat_dis_2020_latlong.dtypes

disasterType                       object
incidentType                       object
declaredCountyArea                 object
state                              object
beginYear                           int64
beginMonth                          int64
beginDay                            int64
incidentBeginDate     datetime64[ns, UTC]
address                            object
latitude                          float64
longitude                         float64
dtype: object

In [22]:
nat_dis_2020_latlong.drop(columns = ['disasterType', 'declaredCountyArea', 'beginMonth', 'beginDay', 'beginYear', 'state'], inplace = True)

In [23]:
nat_dis_2020_latlong.head()

,incidentType,incidentBeginDate,address,latitude,longitude
0,Tornado,2020-03-03 00:01:00+00:00,"Wilson County, TN",36.146666,-86.312000
1,Tornado,2020-03-03 00:01:00+00:00,"Putnam County, TN",36.137987,-85.455378
2,Tornado,2020-03-03 00:01:00+00:00,"Davidson County, TN",36.189724,-86.785786
3,Fire,2020-03-07 08:00:00+00:00,"Beaver County, OK",36.762112,-100.490495
4,Flood,2020-01-10 00:00:00+00:00,"Milwaukee County, WI",43.018033,-87.974087


In [24]:
nat_dis_2020_latlong.to_csv('./../Mapping_Data/natdis_2020_map.csv', index = False)

## COVID-19

In [25]:
covid = pd.read_csv('./../COVID_Data/Historic_data/CCSE/time_series_covid19_confirmed_US.csv')

In [26]:
print(covid.shape)
covid.head(8)

(3261, 117)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,141,141,141,145,145,145,145,145,145,149
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,14,14,14,14,14,14,14,14,14,15
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,1389,1400,1433,1539,1575,1757,1808,1843,1924,1968
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,57,57,57,66,66,66,66,66,66,66
5,84001001.0,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,39,40,43,44,42,45,48,53,53,58
6,84001003.0,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,168,171,174,174,175,181,187,188,189,196
7,84001005.0,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,35,37,37,39,42,43,45,45,47,47


In [27]:
covid = covid.drop(columns = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'])

In [28]:
covid.columns

Index(['Province_State', 'Country_Region', 'Lat', 'Long_', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '4/27/20', '4/28/20', '4/29/20', '4/30/20', '5/1/20', '5/2/20',
       '5/3/20', '5/4/20', '5/5/20', '5/6/20'],
      dtype='object', length=110)

In [51]:
formatted_covid = []
for i, row in covid.iterrows():
    for index, column in enumerate(covid.columns[4:], start = 4):
        formatted_covid.append([row[0], row[1], row[2], row[3], column, row[index]])

In [52]:
formatted_covid = pd.DataFrame(formatted_covid)

In [53]:
formatted_covid.head(2)

,0,1,2,3,4,5
0,American Samoa,US,-14.271,-170.132,1/22/20,0
1,American Samoa,US,-14.271,-170.132,1/23/20,0


In [54]:
formatted_covid.columns = ['State', 'Country', 'Latitude', 'Longitude', 'Date', 'Cases']

In [55]:
formatted_covid.head(2)

,State,Country,Latitude,Longitude,Date,Cases
0,American Samoa,US,-14.271,-170.132,1/22/20,0
1,American Samoa,US,-14.271,-170.132,1/23/20,0


In [56]:
formatted_covid['Date'] = pd.to_datetime(formatted_covid['Date'])

In [57]:
pops = pd.read_csv('../COVID_Data/Population_data/state_population_data.csv')

In [58]:
pops = pops[['NAME', 'POPESTIMATE2019']]

In [59]:
pops = pops.rename(columns={'NAME': 'State', 'POPESTIMATE2019': 'est_population'}) 

In [60]:
US_states = [str(s) for s in us.states.STATES]

In [61]:
pops = pops.loc[pops['State'].isin(US_states)]

In [62]:
pops.set_index('State', inplace=True)

In [63]:
formatted_covid = formatted_covid.join(pops, on='State', how='left')

In [64]:
formatted_covid[formatted_covid['est_population'].isnull()]['State'].value_counts()

District of Columbia        318
Virgin Islands              106
Diamond Princess            106
Guam                        106
American Samoa              106
Grand Princess              106
Northern Mariana Islands    106
Puerto Rico                 106
Name: State, dtype: int64

In [65]:
formatted_covid.dropna(inplace = True)

In [66]:
formatted_covid

,State,Country,Latitude,Longitude,Date,Cases,est_population
530,Alabama,US,32.539527,-86.644082,2020-01-22,0,4903185.0
531,Alabama,US,32.539527,-86.644082,2020-01-23,0,4903185.0
532,Alabama,US,32.539527,-86.644082,2020-01-24,0,4903185.0
533,Alabama,US,32.539527,-86.644082,2020-01-25,0,4903185.0
534,Alabama,US,32.539527,-86.644082,2020-01-26,0,4903185.0
...,...,...,...,...,...,...,...
345661,Utah,US,41.271160,-111.914512,2020-05-02,166,3205958.0
345662,Utah,US,41.271160,-111.914512,2020-05-03,167,3205958.0
345663,Utah,US,41.271160,-111.914512,2020-05-04,167,3205958.0
345664,Utah,US,41.271160,-111.914512,2020-05-05,173,3205958.0


In [67]:
formatted_covid['case_weight'] = formatted_covid['Cases'] / formatted_covid['est_population']

In [68]:
formatted_covid.head()

,State,Country,Latitude,Longitude,Date,Cases,est_population,case_weight
530,Alabama,US,32.539527,-86.644082,2020-01-22,0,4903185.0,0.0
531,Alabama,US,32.539527,-86.644082,2020-01-23,0,4903185.0,0.0
532,Alabama,US,32.539527,-86.644082,2020-01-24,0,4903185.0,0.0
533,Alabama,US,32.539527,-86.644082,2020-01-25,0,4903185.0,0.0
534,Alabama,US,32.539527,-86.644082,2020-01-26,0,4903185.0,0.0


In [69]:
formatted_covid.drop(columns = ['Country', 'est_population'], inplace = True)

In [70]:
formatted_covid.rename(columns = {'case_weight' : 'Cases_per_pop'}, inplace = True)

In [71]:
formatted_covid.sort_values(by = 'Cases_per_pop', ascending = False).head()

,State,Latitude,Longitude,Date,Cases,Cases_per_pop
197583,New York,40.767273,-73.971526,2020-05-06,178351,0.009168
197582,New York,40.767273,-73.971526,2020-05-05,176874,0.009092
197581,New York,40.767273,-73.971526,2020-05-04,175651,0.009029
197580,New York,40.767273,-73.971526,2020-05-03,174331,0.008961
197579,New York,40.767273,-73.971526,2020-05-02,172354,0.008860


In [72]:
formatted_covid.to_csv('./../Mapping_Data/historic_covid_map.csv', index = False)